In [9]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from random import randint
from time import time
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd
import numpy as np
import math
import os
import unicodedata
import string
import time
from functools import wraps
from colorama import Fore
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException

from requests import ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError

import ast

In [10]:
def retry(ExceptionToCheck, tries=20, delay=3, backoff=2, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    :param ExceptionToCheck: the exception to check. may be a tuple of
        exceptions to check
    :type ExceptionToCheck: Exception or tuple
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except ExceptionToCheck:
                    msg = "%s, Retrying in %d seconds..." % (str(ExceptionToCheck), mdelay)
                    if logger:
                        #logger.exception(msg) # would print stack trace
                        logger.warning(msg)
                    else:
                        print(msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry  

In [11]:
result_dataframe = pd.read_csv("./data/state_data.csv", index_col=0)
result_dataframe['Cities'] = result_dataframe['Cities'].apply(lambda x: ast.literal_eval(x))

discover_dataframe = pd.DataFrame({})
restaurant_dataframe = pd.DataFrame({})

In [12]:
result_dataframe = result_dataframe.loc[result_dataframe['State']=='Connecticut']

In [13]:
# keep only 50 states of data
updated_state_data = pd.read_csv(f"./data/us_cities_states_counties.csv", index_col=0)
state=updated_state_data.loc[updated_state_data['st_long']=='Connecticut']
list_2, list_1 = state.city.unique(), result_dataframe['Cities'][6]

cities = np.setdiff1d(list_2,list_1)
# yields the elements in `list_2` that are NOT in `list_1`

In [112]:
class GoogleSpider(object):
  def __init__(self, state, cities):
    """
      Crawl Google search results

      This class is used to crawl Google's search results using requests and BeautifulSoup.
    """
    super().__init__()
    self.discover_dataframe = pd.DataFrame({})
    self.restaurant_dataframe = pd.DataFrame({})
    self.state = state
    self.cities = cities
    self.counter=0
    
  def __driver(self, thread_name):
    """
      Create Selenium Driver
      
      Params:
        counter: str
        changes user data directory every 200th iteration
        eg. counter=counter
    """
    #MACBOOK DATA
    options = webdriver.ChromeOptions()
    options.add_argument("no-sandbox")
    options.add_argument("disable-dev-shm-usage")
    #options.add_argument("headless")
    options.add_argument(f"user-data-dir=/Users/brianphelps/Library/Application Support/Google/Chrome/{thread_name}{self.counter}")
    driver = webdriver.Chrome(executable_path="/Users/brianphelps/Desktop/chromedriver", chrome_options=options)
    
    return driver
    
    
  def __main_places(self, html):
    """
      Crawl Featured Places from Search
      
      This grabs links and lists of featured spots.
    """
    try:
      more_places = html.find(text='Discover more places').parent.parent.parent.find('g-scrolling-carousel').div.div.find_next('div').parent
      for place in more_places:
        discover_title = place.find('g-inner-card').find_all('div')[-1].parent.div.text.strip()
        discover_link = f"google.com{place.a['href']}"

        temp_result = {
          'title': discover_title,
          'link': discover_link,
          'city': [self.city],
          'state': [self.state]
        }
        self.discover_dataframe = self.discover_dataframe.append(temp_result, ignore_index=True)
    except:
      pass
    
  def __picture_data(self,restaurant):
    try:
      restaurant_html = BeautifulSoup(self.restaurant_driver.page_source, 'lxml')
      # define pictures link
      pic_link = restaurant_html.find('div', attrs={"data-attrid":"kc:/location/location:media"}).div.a['href']
      pic_link = f"https://www.google.com{pic_link}"
      print(Fore.GREEN + f"Picture Link: {pic_link}")


      # grab pictures that owner uploaded
      pic_driver = self.__driver('__Picture__')



      @retry((ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
      def serve_pics(link):
        pic_driver.get(link)
      serve_pics(pic_link)
      time.sleep(10)
      pic_driver.execute_script("""
      var divTags = document.getElementsByTagName("div");
      var searchText = "By owner";
      var found;

      for (var i = 0; i < divTags.length; i++) {
        if (divTags[i].textContent == searchText) {
          found = divTags[i];
          found.click();
          break;
        }
      }
      """)
      for y in range(5):
        pic_driver.execute_script("""
        var objDiv = document.querySelector("#gallery > div.widget-pane.widget-pane-visible > div.widget-pane-content.scrollable-y > div > div > div.section-layout.section-scrollbox.scrollable-y.scrollable-show");
        var height = objDiv.scrollHeight;
        objDiv.scrollTop = objDiv.scrollHeight;
        """)
        time.sleep(3)
      # parse picture html
      pics_source = pic_driver.page_source
      pics_html = BeautifulSoup(pics_source, 'lxml')

      pictures = pics_html.find('div', class_='section-layout section-scrollbox scrollable-y scrollable-show').div
      pic_links = []
      for pic in pictures:
        try:
          pic_link = pic.div.a.div.div['style']
          pic_link = pic_link.split("background-image: url")[1].replace("(", "").replace(")", "").replace(";", "").replace('"', '')
          pic_links.append(pic_link)
        except:
          pass
      pictures = pic_links
      print(Fore.GREEN + "Pictures")
    except Exception as e:
      print("Pictures Error")
      print(e)
      pictures = np.nan
    clear_output(wait=True)
    
    try:
      pic_driver.quit()
    except:
      pass
    return pictures

    
  def __restaurant_data(self,idx, restaurant):
    try:
      title = restaurant.a.div.find_next('div').find_next_siblings()[0].div.text
      print(title)

      # RESTAURANT ITEM JS PATH

      self.restaurant_driver.execute_script(f"""
      document.querySelector("#rl_ist0 > div.rl_tile-group > div.rlfl__tls.rl_tls > div:nth-child({idx+1}) > div > div.uMdZh.tIxNaf.mnr-c > div > a > div").click()
      """)
      time.sleep(3)
      restaurant_html = BeautifulSoup(self.restaurant_driver.page_source, 'lxml')
      container = restaurant_html.find_all('span', text=title)[0].parent.parent.parent.parent.parent
    except:
      container = np.nan
      title = np.nan
      pass


    try:
      star_count = container.find('g-review-stars').parent.span.text
      print(Fore.GREEN + f"Star Count: {star_count}")
    except:
      star_count = np.nan
      print(Fore.GREEN + f"Star Count Error")

    try:
      review_count = container.find('g-review-stars').parent.find_all('span')[3].span.a.text
      print(Fore.GREEN + f"Review Count: {review_count}")
    except:
      review_count = np.nan
      print(Fore.GREEN + f"Review Count Error")

    try:
      attrs = []
      attributes = container.find_all('div', class_='TLYLSe')[1].div.find_all('span')
      for attr in attributes:
        attrs.append(attr.text.strip())
      print(Fore.GREEN + f"Attributes: {attrs}")
      attributes = attrs
    except Exception as e:
      attributes = np.nan
      print(e)
      print(Fore.RED + f"Attributes Error")


    try:
      address = restaurant_html.find('div', attrs={"data-attrid":"kc:/location/location:address"}).div.div.find_all('span')[1].text
      print(Fore.GREEN + "Address")
    except:
      address = np.nan
      print(Fore.GREEN + "Address")

    try:
      hours = restaurant_html.find('div', attrs={"data-attrid":"kc:/location/location:hours"}).find_next('table').find_all('tr')
      hours_dict ={}
      for val in hours:
        hours_dict[val.find_all('td')[0].text.strip()]=val.find_all('td')[1].text.strip()
      hours = hours_dict
      print(Fore.GREEN + "Hours")
    except:
      hours = np.nan
      print(Fore.RED + "Hours Error")

    try:
      menu = restaurant_html.find('div', attrs={"data-attrid":"kc:/local:menu"}).find_next('a')['href']
      print(Fore.GREEN + "Menu")
    except:
      menu = np.nan
      print(Fore.RED + "Menu")

    try:
      phone = restaurant_html.find('div', attrs={"data-attrid":"kc:/collection/knowledge_panels/has_phone:phone"}).div.div.find_all('span')[1].span.text.strip()
      print(Fore.GREEN + "Phone")
    except:
      phone = np.nan
      print(Fore.RED + "Phone")

    try:
      website = container.find('a', text='Website')['href']
      print(Fore.GREEN + "Website")
    except:
      website = np.nan
      print(Fore.RED + "Website")
      
    temp = pd.DataFrame({
      'title': [title],
      'star_count': [star_count],
      'review_count': [review_count],
      'attributes': [attributes],
      'address': [address],
      'hours': [hours],
      'menu': [menu],
      'phone': [phone],
      'website':[website],
#       'pictures': [pictures],
      'city': [self.city],
      'state': [self.state]
    })

    return temp

  def search(self):
      """
        Search Google

        Args:
          query (str): The query to search for

        Returns:
          list: The search results
      """
      self.counter = 23
      #define query and link
      for city in self.cities:
        self.city=city
        query = f"{self.city}, {self.state} Restaurants"
        print(query)
        query = query.replace(",", "%2C").replace(" ", "+").replace(" ", "%20")
        main_link = f"https://www.google.com/search?q={query}"

        #create driver
        self.restaurant_driver = self.__driver('__Thread__')
        

        @retry((ReadTimeout, ConnectTimeout, HTTPError, Timeout, ConnectionError), tries=20, delay=2, backoff=2)
        def serve_link(link):
          self.restaurant_driver.get(link)
        serve_link(main_link)

        
        #Catch Captcha
        try:
          source = self.restaurant_driver.page_source
          html = BeautifulSoup(source, 'lxml')
          # TRY TO FIND MORE RESULTS BUTTON
          try:
            result = html.find(text='More places').parent.parent.parent['href']
          except:
            try:
              result = html.find(text='More results').parent.parent.parent['href']
            except:
              result = html.find(text='View all').parent.parent.parent['href']
          link = f"https://www.google.com/{result}"
        except Exception as e:
          print(e)
          print(f"{self.city}, {self.state}")
          #try to find captcha box to see if google is blocking requests
          captcha = html.find('div', class_='recaptcha-checkbox-border')
          if captcha==True:
            breakpoint()
          else:
            self.restaurant_driver.quit()
            continue
            

        try:
          # view all restaurants for location
          WebDriverWait(self.restaurant_driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".MXl0lf"))).click()
        except:
          self.restaurant_driver.quit()
          continue
        #serve_link(link)
        source = self.restaurant_driver.page_source
        html = BeautifulSoup(source, 'lxml')

        time.sleep(3)

        # GRAB MAIN PLACES
        self.__main_places(html)

        #try to click next button
        restaurants_exist = True
        while restaurants_exist:
          #TEST THIS BLOCK
          try:
            #wait until the list of restaurants is visible
            time.sleep(3)
            restaurant_html = BeautifulSoup(self.restaurant_driver.page_source, 'lxml')
            restaurants = restaurant_html.find('div', class_='rl_full-list').div.div.div.div.find_next_siblings()[2].find_next('div').parent
            #grab all restaurant info
            restaurant_length = len(restaurants)
          except Exception as e:
            print(e)
            restaurants_exist = False
            break
            
            
          if restaurants_exist==True:
            for idx, restaurant in enumerate(restaurants):
              #SKIP OVER ADVERTISEMENTS
              
              try:
                if "Why this ad?" in restaurant.a.div.find_next('div').find_next_siblings()[0].div.text:
                  continue
              except:
                pass

              rest_data = self.__restaurant_data(idx, restaurant)
              rest_data['pictures'] = [self.__picture_data(restaurant)]
              self.restaurant_dataframe = self.restaurant_dataframe.append(rest_data, ignore_index=True)
              
            if len(self.restaurant_dataframe)%200==0:
              self.counter+=1
            try:
              restaurant_html = BeautifulSoup(self.restaurant_driver.page_source, 'lxml')
              time.sleep(3)
              WebDriverWait(self.restaurant_driver, 10).until(
                EC.element_to_be_clickable((By.ID, "pnnext"))
              ).click()
            except:
              restaurants_exist = False
        self.discover_dataframe.to_csv("./discover_ct.csv")
        self.restaurant_dataframe.to_csv("./ct_restaurants.csv")
        try:
          self.restaurant_driver.quit()
        except:
          pass
      return self.discover_dataframe, self.restaurant_dataframe

In [ ]:
spider = GoogleSpider(state='Connecticut', cities=list(cities))
disc, rest = spider.search()

Picture Link: https://www.google.com/maps/uv?pb=!1s0x89e6828448b05299:0x59d069e23c28a690!3m1!7e115!4shttps://lh5.googleusercontent.com/p/AF1QipMf2Y4et1ZDa1Ws2Qdl0sVf4H0pT48_5SEVvt2I%3Dw520-h350-n-k-no!5sAbington,+Connecticut+Restaurants+-+Google+Search!15zQ2dJZ0FRPT0&imagekey=!1e10!2sAF1QipMf2Y4et1ZDa1Ws2Qdl0sVf4H0pT48_5SEVvt2I&hl=en
